# SQL 2/3
> Cours NSI Terminale - Thème 2.

# Aller plus loin en SQL

Dans ce classeur, nous poursuivrons l'étude de l'exemple de la base de données de livres de SF et nous verrons des requêtes de sélection avancées ainsi que la manière de fusionner plusieurs tables (Jointures).

Commencez par valider les cellules suivantes afin d'activer la fonctionnalité SQL de Jupyter et de charger la base du TP précédent.

In [2]:
%load_ext sql

%sql sqlite:///livres_db

## Aller plus loin avec SELECT

### Sélection simple

Supposons que l'on veuille lister seulement les noms et prénoms des auteurs nés avant 1900, on utilisera une clause **WHERE** *condition* :

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur from Auteurs WHERE AnneeNaissance < 1900;

Pour obtenir les auteurs prénommés *Jules* :

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur from Auteurs WHERE PrenomAuteur = 'Jules';

Vous voyez que les textes sont délimités par des quotes **'...'**.

***Attention*** : essayez de remplacer **Jules** par **jules** et vous constaterez que la recherche est sensible aux majuscules.

Supposns maintenant que l'on recherche les auteurs dont le prénom commence par **P** : Nous utiliserons une clause *LIKE* de cette manière :

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur from Auteurs WHERE PrenomAuteur LIKE 'P%';

### Sélection multiple
Il est possible de croiser plusieurs critères à l'aide d'opérateurs booleens : **AND** et **OR**. Voici la liste des auteurs français nés après 1900 :

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur from Auteurs WHERE IdLangue = 2 AND AnneeNaissance > 1900;

### Compter le nombre de réponses d'une requête SELECT

Combien y a t-il d'auteurs nés entre 1900 et 1915 ? Vous verrez à l'occation l'utilisation de l'opérateur *BETWEEN* pour tester l'appartenance à un intervalle.

In [ ]:
nbr = %sql SELECT COUNT(*) from Auteurs WHERE AnneeNaissance BETWEEN 1900 AND 1915;

print(nbr)
print("récupérer juste le nombre : ", nbr[0][0])

### Trier les réponses

Nous allons lister tous les auteurs par ordre croissant d'année de naissance

In [ ]:
%sql SELECT * from Auteurs ORDER BY AnneeNaissance;

et par ordre décroissant, on ajoute **DESC** à la fin de la requête

In [ ]:
%sql SELECT * from Auteurs ORDER BY AnneeNaissance DESC;

### Éviter les occurrences multiples

interrogeons la table **Livres** sur les années de publication, rangées par ordre croissant :

In [ ]:
%sql SELECT AnneePubli from Livres ORDER BY AnneePubli;

On constate la présence de quelques doublons

Pour éviter les redondances dans les résultats, on peut rajouter le mot-clé *DISTINCT* juste après le *SELECT* :

In [ ]:
%sql SELECT DISTINCT AnneePubli from Livres ORDER BY AnneePubli;

### A vous de jouer

1. Donner la liste de tous les titres des livres écrits entre 1920 et 1950.
1. Combien y en a t-il ?

In [17]:
reponse_1 = %sql SELECT Titre FROM Livres WHERE AnneePubli BETWEEN 1920 AND 1950 ; 
reponse_1

 * sqlite:///livres_db
Done.


Titre
1984
Le meilleur des mondes
Chroniques martiennes
Les Robots
Ravage
Le monde des A


In [18]:
reponse_2 = %sql SELECT COUNT(*) As nbLivres FROM Livres WHERE AnneePubli BETWEEN 1920 AND 1950; 
reponse_2

 * sqlite:///livres_db
Done.


nbLivres
6


In [19]:
# Vérification des réponses

assert reponse_1[3][0] == 'Les Robots'
assert reponse_2[0][0] == 6

## Requêtes portant sur plusieurs tables

Jusqu'à présent, nos requêtes ne portaient que sur une seule table. Néanmoins notre liste de livres comporte des donées en provenance de plusieurs tables simultanément. Nous allons voir comment effectuer des requêtes pour croiser des données en provenance de plusieurs tables.

In [ ]:
%sql SELECT * FROM Langues, Auteurs

Comme on peut le constater cette requête est peu pertinente car elle affiche toutes les données de chacune des tables sans effectuer de correspondances. La **clé de jointure** apparaît pourtant ici clairement : il s'agit de **id_langue** qui doit permettre de recouper les informations entre les deux tables : il est en effet inutile d'afficher les données pour lesquelles les langues ne correspondent pas entre les deux tables.

### Jointure

La **jonture** consiste à croiser les données de plusieurs tables pour les présenter sous forme d'un seul tableau. On va utiliser ce mécanisme pour afficher clairement la langue de l'auteur plutôt qu'un numéro qui n'est pas forcément très parlant. Nous utiliserons pour cela l'opérateur **JOIN ... ON** :

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur, Langue, AnneeNaissance FROM Auteurs JOIN Langues ON Auteurs.IdLangue = Langues.IdLangue

Les champs sur lesquels faire la jointure **ayant les mêmes noms dans les 2 tables**, cette requête peut aussi être écrite plus simplement en utilisant le mot-clé **USING** ainsi : 

In [ ]:
%sql SELECT NomAuteur, PrenomAuteur, Langue, AnneeNaissance  FROM Auteurs JOIN Langues USING (IdLangue)

### A vous de jouer
En croisant la table **Livres** avec la table **Auteurs**

1. récupérer dans la variable `reponse_1` une liste dont les attributs sont **Titre**, **PrenomAuteur**, **NomAuteur** et **AnneePubli**, triée du plus récent au plus ancien.
1. récupérer dans la variable `reponse_2` une liste dont les attributs sont **Titre**, **PrenomAuteur**, **NomAuteur** et **AnneePubli** écrits en français.

In [5]:
reponse_1 = %sql SELECT Titre, PrenomAuteur, NomAuteur, AnneePubli FROM Livres INNER JOIN Auteurs ON LIvres.IdAuteur=Auteurs.IdAuteur ORDER BY AnneePubli ASC ; 
reponse_1

 * sqlite:///livres_db
Done.


Titre,PrenomAuteur,NomAuteur,AnneePubli
De la Terre à la Lune,Jules,Verne,1865
Le meilleur des mondes,Aldous,Huxley,1931
Ravage,René,Barjavel,1943
Le monde des A,Alfred Elton,Van Vogt,1945
1984,George,Orwell,1949
Chroniques martiennes,Ray,Bradbury,1950
Les Robots,Isaac,Asimov,1950
Fondation,Isaac,Asimov,1951
Fahrenheit 451,Ray,Bradbury,1953
La Fin de l’éternité,Isaac,Asimov,1955


In [ ]:
%sql SELECT Titre, Intitule FROM Livres JOIN RelationsLivreTheme USING (IdLivre) JOIN Themes USING (IdTheme)

### A vous de jouer

Ecrire une requête permettant d'obtenir une liste dont les attributs sont **Titre**, **NomAuteur** et **Langue** triée par ordre croissant de date de naissance de l'auteur.

Vous stockerez le résultat dans une variable `result`

In [14]:
result = %sql SELECT Titre, NomAuteur, Langue FROM Livres INNER JOIN Auteurs ON LIvres.IdAuteur=Auteurs.IdAuteur INNER JOIN Langues ON Langues.IdLangue=Auteurs.IdLangue ORDER BY AnneeNaissance ASC;
result

 * sqlite:///livres_db
Done.


Titre,NomAuteur,Langue
De la Terre à la Lune,Verne,Français
Le meilleur des mondes,Huxley,Anglais
1984,Orwell,Anglais
La nuit des temps,Barjavel,Français
Ravage,Barjavel,Français
La Planète des singes,Boulle,Français
Le monde des A,Van Vogt,Anglais
Dune,Herbert,Anglais
Fondation,Asimov,Anglais
Fahrenheit 451,Bradbury,Anglais


In [16]:
assert result[0] == ('De la Terre à la Lune', 'Verne', 'Français')